### Imports

In [ ]:
import numpy as np
from neuropy.utils.neurons_util import calculate_neurons_acg
import matplotlib.pyplot as plt
import subjects
from neuropy import plotting

### Visualize ACGs

In [ ]:
from neuropy.utils.mathutil import min_max_scaler

sessions = subjects.sd.ratUday1

for s,sess in enumerate(sessions):

    neurons = sess.neurons_stable.get_neuron_type(['pyr','mua'])
    neurons = neurons[neurons.firing_rate<=10]

    frate = neurons.get_binned_spiketrains(bin_size=300).firing_rate
    frate = min_max_scaler(frate,axis=-1)
    t_frate = np.linspace(0,sess.eegfile.duration,frate.shape[1])/3600
    # neurons = sess.neurons
    n_spikes = neurons.n_spikes
    isi = neurons.get_isi()
    ref_spikes = isi[:,:2].sum(axis=1)
    max_isi = isi.max(axis=1)
    cont = ref_spikes/n_spikes
    cont_rel_max = ref_spikes/max_isi


    fp = 0.05  # accepted contamination level
    T = sess.eegfile.duration
    taur = 2e-3
    tauc = 1e-3
    nbadspikes = lambda N: 2 * (taur - tauc) * (N ** 2) * (1 - fp) * fp / T

    expected_violations = [nbadspikes(_) for _ in n_spikes]

    acgs = calculate_neurons_acg(neurons,bin_size=0.001,window_size=0.1)
    t_ccg = np.linspace(-50,50,acgs.shape[1])

    fig = plotting.Fig(grid=(8,8))
    # _,axs = plt.subplots(4,5)
    # axs = axs.reshape(-1)
    for i,acg in enumerate(acgs):
        # ax = axs[i]
        ax = fig.subplot(fig.gs[i])
        ax.plot(t_frate,frate[i])
        ax.set_ylim(0,2)
        ax.set_yticks([])
        fig.toggle_spines(ax,sides=['left'],keep=False)

        axin = ax.inset_axes([0.5,0.6,0.47,0.37])
        axin.plot(t_ccg,acg)
        axin.set_yticks([])
        fig.toggle_spines(axin,sides=['left'],keep=False)

    